# UCSAS Final Code File
## Import Necessary Packages

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from tabulate import tabulate

## Data Cleaning

In [15]:
data_2223 = pd.read_csv("data_2022_2023.csv")
# data_2223.drop(columns=['Unnamed: 14','Unnamed: 15'],inplace=True)
# data_2223=data_2223.dropna()
data_2223['FirstName'] = data_2223['FirstName'].str.strip()
data_2223['LastName'] = data_2223['LastName'].str.strip()
# data_2223[data_2223['FirstName'] == 'Chaopan']

### Cleaning Datetime Values

In [16]:
def get_first_date(date_str):
    # If the date string contains a range (indicated by ' - '), split and take the first part
    if ' - ' in date_str:
        date_str = date_str.split(' - ')[0]
    
    # Now, split by space to separate day, month, and year
    date_parts = date_str.split()
    
    # If the day part contains a range (e.g., '24-27'), take the first day
    day = date_parts[0].split('-')[0] if '-' in date_parts[0] else date_parts[0]
    month = date_parts[1]
    year = date_parts[2] if len(date_parts) == 3 else None
    
    # Reconstruct the date string
    return f'{day} {month} {year}'

# Apply the function to the 'Date' column and convert to datetime
# Specifying dayfirst=True to ensure correct parsing of dates
data_2223['Date'] = pd.to_datetime(data_2223['Date'].apply(get_first_date), dayfirst=True, errors='coerce')

### Cleaning Duplicate Names of Same Athletes

In [17]:
# Make sure names are in correct form and put first/last names in one column
data_2223 = data_2223.applymap(lambda s:s.lower() if type(s) == str else s)
data_2223['FirstName'] = data_2223['FirstName'].str.upper()
data_2223['LastName'] = data_2223['LastName'].str.upper()
data_2223['name'] = data_2223['FirstName'] + ' ' + data_2223['LastName']
data_2223 = data_2223.reindex(columns=['name'] + list(data_2223.columns[:-1]))
#Drop columns that have no name and original first and last name columns
data_2223.dropna(subset=['name'],inplace=True)
data_2223.drop(columns=['LastName','FirstName'],inplace=True)
# Some people's names are displayed in different formats, this resolves that problem

replace_dict = {'TANIGAWA\\\\xa0 KAKERU': 'KAKERU TANIGAWA',
                'CHAOPAN\\\\xa0 LIN': 'CHAOPAN LIN',
                'RUOTENG\\\\xa0 XIAO': 'RUOTENG XIAO',
                'BOHENG\\\\xa0 ZHANG': 'BOHENG ZHANG',
                'KAWAKAMI\\\\xa0 SHOHEI': 'SHOHEI KAWAKAMI',
                'TANIGAWA\\\\xa0 WATARU': 'WATARU TANIGAWA',
                'XINGYU\\\\xa0 LAN': 'XINGYU LAN',
                'KITAZONO\\\\xa0 TAKERU': 'TAKERU KITAZONO',
                'FREDERICK NATHANIEL RICHARD': 'FREDERICK RICHARD',
                'FRED RICHARD': 'FREDERICK RICHARD',
                'SHANE MICHAEL WISKUS': 'SHANE WISKUS',
                'CARLO HOERR': 'CARLO HÖRR',
                'YUL KYUNG TAE MOLDAUER': 'YUL MOLDAUER',
                'ADRIA VERA MORA': 'ADRIA VERA',
                'JORGE RUBIO CERRO': 'JORGE RUBIO',
                'LORAN MUNCK': 'LORAN DE MUNCK',
                'MARTIJN DE VEER': 'MARTIJN DE VEER',
                'DANIEL TUYA PEREZ': 'DANIEL TUYA PEREZ',
                'WOUT TEILLERS': 'WOUT JOHAN ALEXANDER TEILLERS',
                'JERMAIN GRUENBERG': 'JERMAIN GRÜNBERG',
                'IAN SKIRKEY': 'IAN HUNTER SKIRKEY',
                'MEHMET AYBERK': 'MEHMET KOSAK',
                'FRASER GUEST GRIFFITHS': 'FRASER GUEST LYNES',
                'NICOLAU MIR': 'NICOLAU MIR ROSSELLO',
                'AHMET ONDER': 'AHMET ÖNDER',
                'TAYLOR CHRISTOPULOS': 'TAYLOR TROY CHRISTOPULOS',
                'JOSHUA KARNES': 'JOSHUA ANDREW KARNES',
                'KHOI ALEXANDER YOUNG': 'KHOI YOUNG',
                'ZACHARY NATHANIEL CLAY': 'ZACHARY CLAY',
                'YUNUS EMRE GUNDOGDU': 'YUNUS GUNDOGDU',
                'MÉLANIE DE JESUS DOS SANTOS': 'MELANIE DE JESUS DOS SANTOS',
                'SHANE MICHAEL WISKUS': 'SHANE WISKUS',
                'YUL KYUNG TAE MOLDAUER': 'YUL MOLDAUER',
                'SYDNEY LESLIE TURNER': 'SYDNEY TURNER',
                'NOLA RHIANNE MATTHEWS': 'NOLA MATTHEWS',
                'MORGANE OSYSSEK REIMER': 'MORGANE OSYSSEK',
                'SHADE VAN OORSCHOT': 'SHADE VAN OORSCHOT',
                'VERA VAN POL': 'VERA POL',
                'LAURIE DENOMMÉE': 'LAURIE DENOMMEE',
                'LÉA FRANCERIES': 'LEA FRANCERIES',
                'POPPY GRACE STICKLER': 'POPPY STICKLER',
                'NOE SAMUEL SEIFERT': 'NOE SEIFERT',
                'CASSANDRA PAIGE LEE':'CASSANDRA LEE',
                'RUBY EVAN': 'RUBY EVANS',
                'LUCY STANHOPE': 'LUCY GEORGIA STANHOPE',
                'MEGAN PARKER': 'MEGAN ROSE PARKER',
                'AMELIE MORGAN': 'AMELIE ROSE MORGAN',
                'JESSICA GADIROVA': 'JESSICA ROSE GADIROVA',
                'JENNIFER GADIROVA': 'JENNIFER ROSE GADIROVA',
                'ALICE KINSELLA': 'ALICE ROSE KINSELLA',
                'GEORGIA MAE GODWIN': 'GEORGIA GODWIN',
                'GEORGIA ROSE BROWN': 'GEORGIA BROWN',
                'SEVERIN KRANZLMÜLLER': 'SEVERIN KRANZLMULLER',
                'AHMET ÖNDER': 'AHMET ONDER',
                'LORIS FRASCA': 'LORIS FRANCESCO FRASCA',
                'NICOLAU MIR': 'NICOLAU MIR ROSSELLO',
                'ESMÉE BEEKHUIS	': 'ESMEE BEEKHUIS',
                'MARTIJN DE VEER': 'MARTIJN DE VEER',
                'WOUT TEILLERS': 'WOUT JOHAN ALEXANDER TEILLERS',
                'JERMAIN GRUENBERG': 'JERMAIN GRÜNBERG',
                'IAN SKIRKEY': 'IAN HUNTER SKIRKEY',
                'VALENTIN BROSTELLA': 'VALENTINA BROSTELLA',
                'VALENTINA BROSTELLA ARIAS': 'VALENTINA BROSTELLA',
                'DORINA BÖCZÖGÖ': 'DORINA BOCSOGO',
                'JOSSIMAR ORLANDO CALVO MORENO JO': 'JOSSIMAR ORLANDO CALVO MORENO',
                'CHEUK LAM CHARLIE CHAN': 'CHEUK LAM CHAN',
                'JHOSSUA ARIEL CORRALES CASTRO': 'JHOSSUA CORRALES',
                'AGUST INGI DAVIDSSON': 'AGUST DAVIDSSON',
                'ALEJANDRO DE LA CRUZ GATO': 'ALEJANDRO DE LA CRUZ',
                'FABIAN ALEJANDRO BARRIUSO': 'FABIAN BARRIUSO',
                'NATALIA GABRIELA DELGADO LOPEZ': 'NATALIA DELGADO',
                'STELLA LOREN DIAZ': 'STELLA DIAZ',
                'SAMUEL DICK': 'SAM DICK',
                'SAMUAL DICK': 'SAM DICK',
                'JOSE CARLOS ESCANDON': 'JOSE ESCANDON',
                'JOSE CARLOS ESCANDÓN MARÍN':'JOSE ESCANDON',
                'GINNA ESCOBAR BETANCUR': 'GINNA ESCOBAR',
                'KEVIN ESPINOSA CASTILLO': 'KEVIN ESPINOSA',
                'AIDEN MICHAEL FRICK': 'AIDEN FRICK',
                'WILLIAM FU-ALLEN': 'WILLIAM FU ALLEN',
                'WILLIAM FUALLEN': 'WILLIAM FU ALLEN',
                'MAXIMILIANO GALICIA FLORES': 'MAXIMILIANO GALICIA',
                'SEBASTIAN NORBERT GAWRONSKI': 'SEBASTIAN GAWRONSKI',
                'ANDRÉS ESTEBAN GIRÓN ÁLVAREZ':'ANDRES ESTEBAN GIRON ALVAREZ',
                'EDWARD ANDRE GONZALES RIVAS':'EDWARD GONZALES',
                'ALEXA GABRIELA GRANDE FRANCO': 'ALEXA GRANDE',
                'JERMAIN GRÜNBERG': 'JERMAIN GRUNBERG',
                'HILDUR MAJA GUDMUNDSDOTTIR': 'HILDUR GUDMUNDSDOTTIR',
                'NELSON GUILBE MORALES':'NELSON GUILBE',
                'YURI GUIMARÃES':'YURI GUIMARAES',
                'YUNUS GÜNDOGDU': 'YUNUS GUNDOGDU',
                'LILLI LEANNE HABISREUTINGER': 'LILLI HABISREUTINGER',
                'RAKAH AL HARITHI': 'RAKAN ALHARITHI',
                'JOSEPH JUDAH HATOGUAN': 'JOSEPH HATOGUAN',
                'HILLARY HERON SOTO': 'HILLARY HERON',
                'HILLARY ALEXANDRA HERON SOTO': 'HILLARY HERON',
                'LANA RAQUEL HERRERA': 'LANA HERRERA',
                'LANA HERRERA RODRIGUEZ': 'LANA HERRERA',
                'AINHOA SOFIA HERRERO LUGO': 'AINHOA HERRERO',
                'VINZENZ JOHANN HOCK': 'VINZENZ HOCK',
                'VINZENZ HOECK': 'VINZENZ HOCK',
                'CARLO HÖRR': 'CARLO HORR',
                'YEN-CHANG HUANG': 'YEN CHANG HUANG',
                'YUAN-HSI HUNG': 'YUAN HSI HUNG',
                'ELISA HÄMMERLE': 'ELISA HAMMERLE',
                'KARL IDESJÖ': 'KARL IDESJO',
                'KARL IDESJOE': 'KARL IDESJO',
                'KARLA ANDREA NAVAS BOYD': 'KARLA NAVAS',
                'KARLA NAVAS BOYD': 'KARLA NAVAS',
                'NICOLE IRIBARNE APARICIO': 'NICOLE IRIBARNE',
                'JULIAN EZEQUIEL JATO': 'JULIAN JATO',
                'YO-SEOP JEON': 'YO SEOP JEON',
                'YOSEOP JEON': 'YO SEOP JEON',
                'DILAN ANDRES JIMENEZ GIRALDO': 'DILAN JIMENEZ',
                'JOSUE JUAREZ JUAREZ': 'JOSUE JUAREZ',
                'AMAYA SITHUMINI KALUKOTTAGE': 'AMAYA KALUKOTTAGE',
                'TARMO TUOMAS KANERVA': 'TARMO KANERVA',
                'MOHAMAD KHALIL': 'MOHAMED KHALIL',
                'MOHAMED KHALIL JENDOUBI': 'MOHAMED KHALIL',
                'SAARA KATARIINA KOKKO': 'SAARA KOKKO',
                'SHANTÉ KOTI': 'SHANTE KOTI',
                'SEVERIN KRANZLMUELLER': 'SEVERIN KRANZLMULLER',
                'MARGRET LEA KRISTINSDOTTIR': 'MARGRET KRISTINSDOTTIR',
                'CARINA KRÖLL': 'CARINA KROLL',
                'EETU KUJANPÄÄ': 'EETU KUJANPAA',
                'HANSA GAYASHAN KUMARASINGHEGE': 'HANSA KUMARASINGHEGE',
                'HANSHA KUMARASINGHEGE': 'HANSA KUMARASINGHEGE',
                'KYLEE ANN KVAMME': 'KYLEE KVAMME',
                'ANNA-LENA KÖNIG': 'ANNA LENA KONIG',
                'PIN LAI': 'PIN JU LAI',
                'PIN-JU LAI': 'PIN JU LAI',
                'DAIRA GISELL LAMADRID': 'DAIRA LAMADRID',
                'CHIH LEE': 'CHIH KAI LEE',
                'CHIH-KAI LEE': 'CHIH KAI LEE',
                'MAN HIN FRANKIE LEE': 'MAN HIN LEE',
                'MILCA ANDREINA LEON ANDRADE': 'MILCA LEON',
                'LUCIANO MAURICIO LETELIER': 'LUCIANO LETELIER',
                'YI-CHUN LIAO': 'YI CHUN LIAO',
                'GUAN LIN': 'GUAN LIN LIN',
                'GUAN-YI LIN': 'GUAN YI LIN',
                'YI LIN': 'YI LIN LIN',
                'YI-CHEN LIN': 'YI CHEN LIN',
                'MAIA LLACER SIRERA': 'MAIA LLACER',
                'SARA SOFIA LOIKAS': 'SARA LOIKAS',
                'NOÉMIE LOUON': 'NOEMIE LOUON',
                'THANH TÙNG LÊ': 'THANH TUNG LE',
                'JULIE MADSØ': 'JULIE MADSO',
                'JULIE MADSOE': 'JULIE MADSO',
                'EMMA LEONIE MALEWSKI': 'EMMA MALEWSKI',
                'JEA BRITTANY MARACHA': 'JEA MARACHA',
                'RAZVAN-DENIS MARCU': 'RAZVAN MARCU',
                'RAZVAN DENIS MARC': 'RAZVAN MARCU',
                'ANTONIA MARIHUAN RUBIO': 'ANTONIA MARIHUAN',
                'CLAY MASONSTEPHENS': 'CLAY MASON STEPHENS',
                'RHYS MC CLENAGHAN': 'RHYS MCCLENAGHAN',
                'LORENA MEDINA COBOSS': 'LORENA MEDINA',
                'DIMITRIJS MICKEVICS': 'DMITRIJS TALALAYEV',
                'TOMA ROLAND MODOIANU ZSEDER': 'TOMA MODOIANU ZSEDER',
                'TOMA MODOIANU-ZSEDER': 'TOMA MODOIANU ZSEDER',
                'CHIARA SUMMER MOISZI': 'CHIARA MOISZI',
                'MALLA ALEXANDRA MONTELL': 'MALLA MONTELL',
                'FRANCINY MORALES BARQUERO': 'FRANCINY MORALES',
                'ALEXA CITLALI MORENO MEDINA': 'ALEXA MORENO',
                'ALEXA MORENO MEDINA': 'ALEXA MORENO',
                'SASIWIMION MUEANGPHUANG': 'SASIWIMON MUEANGPHUANG',
                'SANI MÄKELÄ': 'SANI MAKELA',
                'SANI MAKELAE': 'SANI MAKELA',
                'ALISSA MÖRZ': 'ALISSA MORZ',
                'ALISSA MORZE': 'ALISSA MORZ',
                'CHARLIZE MÖRZ': 'CHARLIZE MORZ',
                'CHARLIZE MORZE': 'CHARLIZE MORZ',
                'JANOAH MÜLLER': 'JANOAH MULLER',
                'KARLA NAVAS BOYD': 'KARLA NAVAS',
                'KARLA ANDREA NAVAS BOYD': 'KARLA NAVAS',
                'ANNALISE BECCA NEWMAN ACHEE': 'ANNALISE NEWMAN ACHEE',
                'VAN KHANH PHONG NGUYEN': 'VAN KHANH NGUYEN',
                'AUDRYS NIN REYES': 'AUDRYS NIN',
                'ISAAC NUÑEZ': 'ISAAC NUNEZ',
                'DAGUR KARI OLAFSSON': 'DAGUR OLAFSSON',
                'AHMET ÖNDER': 'AHMET ONDER',
                'ARETI PARASKEVI PAGONI': 'ARETI PAGONI',
                'ANANYA BELLE PATANAKUL': 'ANANYA PATANAKUL',
                'DMITRY PATANIN': 'DMITRIY PATANIN',
                'LUCIA CAROLINA PAULINO LOPEZ': 'LUCIA PAULINO',
                'ALAIS NATASHA PEREA PONCE': 'ALAIS PEREA',
                'ANDRES JOSUE PEREZ GINEZ': 'ANDRES PEREZ',
                'ANDRES JOSUE PEREZ GINES': 'ANDRES PEREZ',
                'CURRAN MICHAEL PHILLIPS': 'CURRAN PHILLIPS',
                'ANYA KAELIN PILGRIM': 'ANYA PILGRIM',
                'JULIANA MARIA PINEDA': 'JULIANA PINEDA',
                'ERIN GIANNA PINDER': 'ERIN PINDER',
                'MAKARENA DAISY PINTO ADASME': 'MAKARENA PINTO ADASME',
                'JABIEL DE JESUS POLANCO ACOSTA': 'JABIEL POLANCO',
                'ANGELISSA PONCE VILLALPANDO': 'ANGELISSA PONCE',
                'PABLO HAROLD POZO DECOS': 'PABLO POZO',
                'LEONARD PRÜGEL': 'LEONARD PRUGEL',
                'JIMI PÄIVÄNEN': 'JIMI PAIVANEN',
                'JIMI PÄVÄNEN': 'JIMI PAIVANEN',
                'ALBERTO PÉREZ FERNÁNDEZ': 'ALBERTO PEREZ',
                'LEA MARIE QUAAS': 'LEA QUAAS',
                'AL-HARITH RAKAN': 'AL HARITH RAKAN',
                'JEORDY RAMIREZ CASTRO': 'JEORDY RAMIREZ',
                'PAULA RAYA I ARTIGAS': 'PAULA RAYA',
                'MICHAEL JAMES REID': 'MICHAEL REID',
                'DIETMAR V. REINHARDT CODINA': 'DIETMAR REINHARDT',
                'ORIOL RIFA PEDRENO': 'ORIOL RIFA',
                'JOSCELYN MICHELLE ROBERSON': 'JOSCELYN ROBERSON',
                'ANELENA RODRIGUEZ JOHANNING': 'ANELENA RODRIGUEZ',
                'RACHEL RODRIGUEZ MIRANDA': 'RACHEL RODRIGUEZ',
                'SARAI RODRÍGUEZ GARCÍA': 'SARAI RODRIGUEZ GARCIA',
                'KEIRA ROLSTON-LARKING': 'KEIRA ROLSTON LARKING',
                'PAOLA MASSIEL RUANO BARAHONA': 'PAOLA RUANO',
                'LÉO SALADINO': 'LEO SALADINO',
                'DEBORAH MERSEDES SALMINA ARROYO': 'DEBORAH SALMINA',
                'PATRICK SAMPAIO CORREA': 'PATRICK SAMPAIO',
                'AHTZIRI VIRIDIANA SANDOVAL': 'AHTZIRI SANDOVAL',
                'GÖKSU ÜCTAS SANLI': 'GOKSU UCTAS SANLI',
                'FRANCHESCA ANTONELLA SANTI': 'FRANCHESCA SANTI',
                'ROCIO SELENE SAUCEDO': 'ROCIO SAUCEDO',
                'PAULINE SCHÄFER': 'PAULINE SCHAFER',
                'PAULINE SCHAEFER BETZ': 'PAULINE SCHAFER',
                'KARINA SCHÖNMAIER': 'KARINA SCHONMAIER',
                'KARINA SCHOENMAIER': 'KARINA SCHONMAIER',
                'YU SHIAO': 'YU JAN SHIAO',
                'YU-JAN SHIAO': 'YU JAN SHIAO',
                'JOOA SILLANPÄÄ': 'JOOA SILLANPAA',
                'PEDER FUNDERUD SKOGVANG': 'PEDER SKOGVANG',
                'KIPLIN MORRISH SMITH': 'KIPLIN SMITH',
                'SEBASTIÁN ANDRÉS SUE DOMÍNGUEZ': 'SEBASTIAN SUE',
                'DOMINIC DANIEL TAMSEL': 'DOMINIC TAMSEL',
                'CHIA-HUNG TANG': 'CHIA HUNG TANG',
                'DERIN TANRIYASÜKÜR': 'DERIN TANRIYASUKUR',
                'JONAS INGI THORISSON': 'JONAS THORISSON',
                'JANN GWYNN TIMBANG': 'JANN TIMBANG',
                'HUA TING': 'HUA TIEN TING',
                'HUA-TIEN TING': 'HUA TIEN TING',
                'ADICKXON TREJO BASALO': 'ADICKXON TREJO',
                'ADICKXON GABRIEL TREJO BASALO': 'ADICKXON TREJO',
                'WEI TSENG': 'WEI SHENG TSENG',
                'WEI-SHENG TSENG': 'WEI SHENG TSENG',
                'JULIANE TØSSEBRO': 'JULIANE TOSSEBRO',
                'JULIANE TOESSEBRO': 'JULIANE TOSSEBRO',
                'JOHNNY ADRIAN VALENCIA ZAMBRANO': 'JOHNNY ADRIAN VALENCIA',
                'YISETH ELIANA VALENZUELA ASTUDILLO': 'YISETH VALENZUELA',
                'KIM WANSTRÖM': 'KIM WANSTROM',
                'KIM VANSTROEM': 'KIM VANSTROM',
                'IGNACIO JAVIER VARAS': 'IGNACIO VARAS',
                'SERGIO ANDRES VARGAS RINCON': 'SERGIO VARGAS',
                'DIANA STEPHANY VASQUEZ SEOANE': 'DIANA VASQUEZ',
                'PABLO NATANAEL VELASQUEZ CANDRAY': 'PABLO VELASQUEZ',
                'DANIEL ANGEL VILLAFANE': 'DANIEL VILLAFANE',
                'DANIEL VILLAFAÑE': 'DANIEL VILLAFANE',
                'YOHENDRY VILLAVERDE MEDEROS': 'YOHENDRY VILLAVERDE',
                'MARIA JOSE VILLEGAS JIMENEZ': 'MARIA VILLEGAS',
                'TISHA MANOUK GIJS VOLLEMAN': 'TISHA VOLLEMAN',
                'ELEL DILIZA WAHRMANN BAKER': 'ELEL WAHRMANN BAKER',
                'MAX WHITLOCK OBE': 'MAX WHITLOCK',
                'JOSHUA JACK WILLIAMS MEEHAN': 'JOSHUA JACK WILLIAMS',
                'HIU YING ANGEL WONG': 'HIU YING WONG',
                'EMMA EN LIN YAP': 'EMMA YAP',
                'KORKEM YEROBSSYNKYZY': 'KORKEM YERBOSSYNKYZY',
                'CARLOS EDRIEL YULO': 'CARLOS YULO',
                'SAMUEL ZAKUTNEY': 'SAM ZAKUTNEY',              
                
}


data_2223['name'] = data_2223['name'].replace(replace_dict)

C:\Users\janys\AppData\Local\Temp\ipykernel_23576\2796125885.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_2223 = data_2223.applymap(lambda s:s.lower() if type(s) == str else s)


In [ ]:
pd.DataFrame(data_2223['name'].unique().tolist()).to_csv('names.csv')

### Make Key Columns Capitalized

In [19]:
data_2223['Apparatus'] = data_2223['Apparatus'].str.upper()
data_2223['Country'] = data_2223['Country'].str.upper()

### Merge GBR countries

In [21]:
country_replace_dict = {
    "ENG": "GBR",
    "SCO": "GBR",
    "WAL": "GBR",
    "NIR": "GBR"
}
data_2223['Country'] = data_2223['Country'].replace(country_replace_dict)


### Separate Original DataFrame by Gender

In [24]:
men_raw = data_2223[data_2223['Gender']=='m']
women_raw = data_2223[data_2223['Gender']=='w']

In [25]:
men_raw.to_csv('men_raw_1_14_2.csv')
women_raw.to_csv('women_raw_1_14_2.csv')

### Create a New DataFrame for Athelete's Individual Data (Women)

In [26]:
import pandas as pd

# Assuming women_raw and apparatus_map are already defined
apparatus_map = {'VT': 'VT', 'VT1': 'VT', 'VT2': 'VT'}
# Replace Apparatus names according to the mapping
women_raw['Apparatus_Combined'] = women_raw['Apparatus'].replace(apparatus_map)

# Group by 'name', 'Country', 'Apparatus_Combined' and calculate various statistics
grouped = women_raw.groupby(['name', 'Country', 'Apparatus_Combined'])
grouped_mean = grouped['Score'].mean()
grouped_std = grouped['Score'].std()
grouped_count = grouped['Score'].count()
grouped_d_mean = grouped['D_Score'].mean()
grouped_d_std = grouped['D_Score'].std()
grouped_e_mean = grouped['E_Score'].mean()
grouped_e_std = grouped['E_Score'].std()

# Unstack groupby results
mean_scores_df = grouped_mean.unstack(level='Apparatus_Combined')
std_scores_df = grouped_std.unstack(level='Apparatus_Combined')
count_scores_df = grouped_count.unstack(level='Apparatus_Combined')
d_mean_scores_df = grouped_d_mean.unstack(level='Apparatus_Combined')
d_std_scores_df = grouped_d_std.unstack(level='Apparatus_Combined')
e_mean_scores_df = grouped_e_mean.unstack(level='Apparatus_Combined')
e_std_scores_df = grouped_e_std.unstack(level='Apparatus_Combined')

# Flatten the MultiIndex on the columns and rename them
mean_scores_df.columns = [f'{col}_mean_score' for col in mean_scores_df.columns]
std_scores_df.columns = [f'{col}_std' for col in std_scores_df.columns]
count_scores_df.columns = [f'{col}_count' for col in count_scores_df.columns]
d_mean_scores_df.columns = [f'{col}_D_mean_score' for col in d_mean_scores_df.columns]
d_std_scores_df.columns = [f'{col}_D_std' for col in d_std_scores_df.columns]
e_mean_scores_df.columns = [f'{col}_E_mean_score' for col in e_mean_scores_df.columns]
e_std_scores_df.columns = [f'{col}_E_std' for col in e_std_scores_df.columns]

# Calculate the standard deviation of all scores for each athlete
all_scores_std = women_raw.groupby(['name', 'Country'])['Score'].std()

# Join the new statistics to the combined DataFrame
women_individual = pd.concat([
    mean_scores_df, std_scores_df, count_scores_df,
    d_mean_scores_df, d_std_scores_df, e_mean_scores_df, e_std_scores_df
], axis=1)

# Reset the index to include 'Country' and merge with the standard deviation of all scores
women_individual.reset_index(inplace=True)
women_individual = women_individual.merge(all_scores_std.rename('all_scores_std'), on=['name', 'Country'])

# Define the apparatus list for women
apparatus_list = ['UB', 'BB', 'FX', 'VT']

# Extract the base apparatus names from the mean score columns
apparatus_base_names = [col.replace('_mean_score', '') for col in women_individual.columns if '_mean_score' in col]
new_column_order = ['name', 'Country']  # Start with the 'name' and 'Country' columns

# Loop over the base names and append the desired columns in the specified order
for base_name in apparatus_list:
    new_column_order.extend([
        f'{base_name}_mean_score', f'{base_name}_std', f'{base_name}_count',
        f'{base_name}_D_mean_score', f'{base_name}_D_std',
        f'{base_name}_E_mean_score', f'{base_name}_E_std',
        f'{base_name}_rank'  # Adding rank column in the loop
    ])

# Add the rank column after the E_std column for each apparatus
for apparatus in apparatus_list:
    rank_column_name = f'{apparatus}_rank'
    women_individual[rank_column_name] = women_individual[f'{apparatus}_mean_score'].rank(ascending=False)

# Reorder the columns
new_column_order = new_column_order + ['all_scores_std']
women_individual = women_individual[new_column_order]

# Fill NA values and round off decimals
women_individual.fillna(0, inplace=True)
women_individual = women_individual.round(2)

# Calculate additional statistics
women_individual['Standardized_Score_Sum'] = women_individual[[f'{app}_mean_score' for app in apparatus_list]].sum(axis=1)
women_individual['Total_Appearances'] = women_individual[[f'{app}_count' for app in apparatus_list]].sum(axis=1)
count_columns = [col for col in women_individual.columns if '_count' in col]
women_individual['apparatus_participated'] = women_individual[count_columns].apply(lambda row: (row > 0).sum(), axis=1)
women_individual['mean_score_of_all'] = sum(women_individual[f'{app}_mean_score'] * women_individual[f'{app}_count'] for app in apparatus_list) / women_individual['Total_Appearances']
women_individual['mean_score_of_all'] = women_individual['mean_score_of_all'].round(2)

# Sorting and displaying the top 5 athletes
women_individual.sort_values('Standardized_Score_Sum', ascending=False, inplace=True)
women_individual['all_scores_std'] = women_individual['all_scores_std'].round(2)
women_individual.head(5)


C:\Users\janys\AppData\Local\Temp\ipykernel_23576\4219383285.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  women_raw['Apparatus_Combined'] = women_raw['Apparatus'].replace(apparatus_map)


,name,Country,UB_mean_score,UB_std,UB_count,UB_D_mean_score,UB_D_std,UB_E_mean_score,UB_E_std,UB_rank,...,VT_D_mean_score,VT_D_std,VT_E_mean_score,VT_E_std,VT_rank,all_scores_std,Standardized_Score_Sum,Total_Appearances,apparatus_participated,mean_score_of_all
675,SIMONE BILES,USA,14.26,0.18,7.0,6.00,0.00,8.26,0.18,9.0,...,5.96,0.42,9.26,0.37,1.0,0.41,58.71,30.0,4,14.70
608,REBECA ANDRADE,BRA,14.23,0.56,11.0,6.11,0.14,8.12,0.47,10.0,...,5.27,0.68,9.40,0.29,2.0,0.78,56.60,44.0,4,14.21
668,SHILESE JONES,USA,14.51,0.49,15.0,6.29,0.08,8.22,0.42,6.0,...,5.00,0.00,9.31,0.16,4.0,0.69,55.82,53.0,4,13.96
397,KONNOR MCCLAIN,USA,13.27,0.80,3.0,5.60,0.20,7.67,0.60,66.0,...,5.00,0.00,9.28,0.16,5.0,0.67,55.68,12.0,4,13.92
336,JESSICA ROSE GADIROVA,GBR,13.39,0.44,12.0,5.58,0.14,7.81,0.41,55.0,...,4.98,0.28,8.98,0.36,19.0,0.56,54.67,58.0,4,13.72


In [27]:
women_individual.to_csv('women_1_14_3.csv')

### Create a New DataFrame for Athelete's Individual Data (Men)

In [29]:
import pandas as pd

# Assuming men_raw and apparatus_map are already defined

# Replace Apparatus names according to the mapping
men_raw['Apparatus_Combined'] = men_raw['Apparatus'].replace(apparatus_map)

# Group by 'name', 'Country', 'Apparatus_Combined' and calculate various statistics
grouped = men_raw.groupby(['name', 'Country', 'Apparatus_Combined'])
grouped_mean = grouped['Score'].mean()
grouped_std = grouped['Score'].std()
grouped_count = grouped['Score'].count()
grouped_d_mean = grouped['D_Score'].mean()
grouped_d_std = grouped['D_Score'].std()
grouped_e_mean = grouped['E_Score'].mean()
grouped_e_std = grouped['E_Score'].std()

# Unstack groupby results
mean_scores_df = grouped_mean.unstack(level='Apparatus_Combined')
std_scores_df = grouped_std.unstack(level='Apparatus_Combined')
count_scores_df = grouped_count.unstack(level='Apparatus_Combined')
d_mean_scores_df = grouped_d_mean.unstack(level='Apparatus_Combined')
d_std_scores_df = grouped_d_std.unstack(level='Apparatus_Combined')
e_mean_scores_df = grouped_e_mean.unstack(level='Apparatus_Combined')
e_std_scores_df = grouped_e_std.unstack(level='Apparatus_Combined')

# Flatten the MultiIndex on the columns and rename them
mean_scores_df.columns = [f'{col}_mean_score' for col in mean_scores_df.columns]
std_scores_df.columns = [f'{col}_std' for col in std_scores_df.columns]
count_scores_df.columns = [f'{col}_count' for col in count_scores_df.columns]
d_mean_scores_df.columns = [f'{col}_D_mean_score' for col in d_mean_scores_df.columns]
d_std_scores_df.columns = [f'{col}_D_std' for col in d_std_scores_df.columns]
e_mean_scores_df.columns = [f'{col}_E_mean_score' for col in e_mean_scores_df.columns]
e_std_scores_df.columns = [f'{col}_E_std' for col in e_std_scores_df.columns]

# Calculate the standard deviation of all scores for each athlete
all_scores_std = men_raw.groupby(['name', 'Country'])['Score'].std()

# Join the new statistics to the combined DataFrame
men_individual = pd.concat([
    mean_scores_df, std_scores_df, count_scores_df,
    d_mean_scores_df, d_std_scores_df, e_mean_scores_df, e_std_scores_df
], axis=1)

# Reset the index to include 'Country' and merge with the standard deviation of all scores
men_individual.reset_index(inplace=True)
men_individual = men_individual.merge(all_scores_std.rename('all_scores_std'), on=['name', 'Country'])

# Define the apparatus list
apparatus_list = ['FX', 'HB', 'PB', 'PH', 'SR', 'VT']

# Extract the base apparatus names from the mean score columns
# Note: Changed from 'women_individual' to 'men_individual'
apparatus_base_names = [col.replace('_mean_score', '') for col in men_individual.columns if '_mean_score' in col]
new_column_order = ['name', 'Country']  # Start with the 'name' and 'Country' columns

# Loop over the base names and append the desired columns in the specified order
for base_name in apparatus_list:
    new_column_order.extend([
        f'{base_name}_mean_score', f'{base_name}_std', f'{base_name}_count',
        f'{base_name}_D_mean_score', f'{base_name}_D_std',
        f'{base_name}_E_mean_score', f'{base_name}_E_std',
        f'{base_name}_rank'  # Adding rank column in the loop
    ])

# Add the rank column after the E_std column for each apparatus
for apparatus in apparatus_list:
    rank_column_name = f'{apparatus}_rank'
    men_individual[rank_column_name] = men_individual[f'{apparatus}_mean_score'].rank(ascending=False)

# Reorder the columns
new_column_order = new_column_order + ['all_scores_std']
men_individual = men_individual[new_column_order]

# Fill NA values and round off decimals
men_individual.fillna(0, inplace=True)
men_individual = men_individual.round(2)

# Calculate additional statistics
men_individual['Standardized_Score_Sum'] = men_individual[[f'{app}_mean_score' for app in apparatus_list]].sum(axis=1)
men_individual['Total_Appearances'] = men_individual[[f'{app}_count' for app in apparatus_list]].sum(axis=1)
count_columns = [col for col in men_individual.columns if '_count' in col]
men_individual['apparatus_participated'] = men_individual[count_columns].apply(lambda row: (row > 0).sum(), axis=1)
men_individual['mean_score_of_all'] = sum(men_individual[f'{app}_mean_score'] * men_individual[f'{app}_count'] for app in apparatus_list) / men_individual['Total_Appearances']
men_individual['mean_score_of_all'] = men_individual['mean_score_of_all'].round(2)

# Sorting and displaying the top 5 athletes
men_individual.sort_values('Standardized_Score_Sum', ascending=False, inplace=True)
men_individual['all_scores_std'] = men_individual['all_scores_std'].round(2)
men_individual.head(5)


C:\Users\janys\AppData\Local\Temp\ipykernel_23576\412322557.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  men_raw['Apparatus_Combined'] = men_raw['Apparatus'].replace(apparatus_map)


,name,Country,FX_mean_score,FX_std,FX_count,FX_D_mean_score,FX_D_std,FX_E_mean_score,FX_E_std,FX_rank,...,VT_D_mean_score,VT_D_std,VT_E_mean_score,VT_E_std,VT_rank,all_scores_std,Standardized_Score_Sum,Total_Appearances,apparatus_participated,mean_score_of_all
112,BOHENG ZHANG,CHN,14.43,0.26,8.0,6.02,0.14,8.42,0.18,4.0,...,5.60,0.00,8.97,0.50,14.0,0.76,86.56,46.0,6,14.43
496,KAZUMA KAYA,JPN,13.84,0.49,6.0,5.70,0.00,8.16,0.50,49.0,...,5.60,0.00,8.80,0.17,42.0,0.38,85.41,35.0,6,14.27
934,WATARU TANIGAWA,JPN,13.94,0.50,4.0,5.70,0.08,8.27,0.51,36.0,...,5.64,0.33,9.02,0.21,8.0,0.52,85.27,30.0,6,14.34
173,CONG SHI,CHN,13.87,0.03,3.0,5.53,0.23,8.33,0.20,47.0,...,5.60,0.00,8.30,0.66,190.0,0.59,84.97,23.0,6,14.21
181,DAIKI HASHIMOTO,JPN,14.26,0.50,10.0,5.95,0.08,8.32,0.45,11.0,...,5.60,0.00,9.13,0.28,9.0,1.55,84.66,51.0,6,14.15


In [30]:
men_individual.to_csv('men_1_14_3.csv')

## Functions for Preliminary Analysis

In [11]:
# Calculate the mean and max scores for each apparatus
FX_mean = round(men_raw[men_raw['Apparatus']=='FX']['Score'].mean(),2)
PB_mean = round(men_raw[men_raw['Apparatus']=='PB']['Score'].mean(),2)
HB_mean = round(men_raw[men_raw['Apparatus']=='HB']['Score'].mean(),2)
SR_mean = round(men_raw[men_raw['Apparatus']=='SR']['Score'].mean(),2)
VT_mean = round(men_raw[men_raw['Apparatus']=='VT']['Score'].mean(),2)
PH_mean = round(men_raw[men_raw['Apparatus']=='PH']['Score'].mean(),2)
# Mean for women
women_FX_mean = round(women_raw[women_raw['Apparatus']=='FX']['Score'].mean(),2)
women_UB_mean = round(women_raw[women_raw['Apparatus']=='UB']['Score'].mean(),2)
women_BB_mean = round(women_raw[women_raw['Apparatus']=='BB']['Score'].mean(),2)
women_VT_mean = round(women_raw[women_raw['Apparatus']=='VT']['Score'].mean(),2)

average_scores = {
    'FX_mean_score': FX_mean,
    'HB_mean_score': HB_mean,
    'PB_mean_score': PB_mean,
    'PH_mean_score': PH_mean,
    'SR_mean_score': SR_mean,
    'VT_mean_score': VT_mean
}

women_average_scores = {
    'FX_mean_score': women_FX_mean,
    'UB_mean_score': women_UB_mean,
    'BB_mean_score': women_BB_mean,
    'VT_mean_score': women_VT_mean
}
# Maximum for men 
FX_max = men_raw[men_raw['Apparatus'] == 'FX']['Score'].max()
PB_max = men_raw[men_raw['Apparatus'] == 'PB']['Score'].max()
HB_max = men_raw[men_raw['Apparatus'] == 'HB']['Score'].max()
SR_max = men_raw[men_raw['Apparatus'] == 'SR']['Score'].max()
VT_max = men_raw[men_raw['Apparatus'] == 'VT']['Score'].max()
PH_max = men_raw[men_raw['Apparatus'] == 'PH']['Score'].max()
# Maximum for women
women_FX_max = women_raw[women_raw['Apparatus'] == 'FX']['Score'].max()
women_UB_max = women_raw[women_raw['Apparatus'] == 'UB']['Score'].max()
women_BB_max = women_raw[women_raw['Apparatus'] == 'BB']['Score'].max()
women_VT_max = women_raw[women_raw['Apparatus'] == 'VT']['Score'].max()

max_scores = {
    'FX_max_score': FX_max,
    'PB_max_score': PB_max,
    'HB_max_score': HB_max,
    'SR_max_score': SR_max,
    'VT_max_score': VT_max,
    'PH_max_score': PH_max
}

women_max_scores = {
    'FX_max_score': women_FX_max,
    'UB_max_score': women_UB_max,
    'BB_max_score': women_BB_max,
    'VT_max_score': women_VT_max
}

def get_men_athelete_info(name,table=True,plot=True):
    # Select the row for the player by name
    player_row = men_individual[men_individual['name'] == name]
    # Define the columns to keep
    columns_to_keep = [
        'all_scores_std', 'Standardized_Score_Sum', 'Total_Appearances', 
        'apparatus_participated', 'mean_score_of_all'
    ]

    player_info = player_row[columns_to_keep].T
    print(tabulate(player_info, headers='keys', tablefmt='pretty'))
    
    if table==True:
        tabular_data = {
        'Apparatus': ['Floor Exercise', 'Horizontal Bar', 'Parallel Bar', 'Pomel Horse', 'Still Ring', 'Vault'],
        'Mean Score': [player_row['FX_mean_score'].item(), player_row['HB_mean_score'].item(), 
                       player_row['PB_mean_score'].item(), player_row['PH_mean_score'].item(), 
                       player_row['SR_mean_score'].item(), player_row['VT_mean_score'].item()],
        'Standard Deviation': [player_row['FX_std'].item(), player_row['HB_std'].item(), 
                               player_row['PB_std'].item(), player_row['PH_std'].item(), 
                               player_row['SR_std'].item(), player_row['VT_std'].item()],
        'Appearances': [player_row['FX_count'].item(), player_row['HB_count'].item(), 
                        player_row['PB_count'].item(), player_row['PH_count'].item(), 
                        player_row['SR_count'].item(), player_row['VT_count'].item()],
        'Rank': [player_row['FX_rank'].item(), player_row['HB_rank'].item(), 
                 player_row['PB_rank'].item(), player_row['PH_rank'].item(), 
                 player_row['SR_rank'].item(), player_row['VT_rank'].item()]
        }
        tabular_df = pd.DataFrame(tabular_data)
        print('Statistics of', name, player_row['Country'].item())
        print(tabulate(tabular_df, headers='keys', tablefmt='pretty', showindex=False))


    if plot == True: 
        # Define the columns for mean and max scores
        score_columns = ['FX_mean_score', 'HB_mean_score', 'PB_mean_score', 'PH_mean_score', 'SR_mean_score', 'VT_mean_score']
        max_score_columns = ['FX_max_score', 'HB_max_score', 'PB_max_score', 'PH_max_score', 'SR_max_score', 'VT_max_score']
        columns = ['Floor Exercise', 'Horizontal Bar', 'Parallel Bar', 'Pomel Horse', 'Still Ring', 'Vault']
        
        # Extract the individual's mean scores
        individual_scores = player_row[score_columns].values.flatten().tolist()
        individual_scores += individual_scores[:1]  # Repeat the first score at the end to close the circle

        # Add the average scores for comparison
        average_scores_list = [average_scores[col] for col in score_columns]
        average_scores_list += average_scores_list[:1]  # Repeat the first score at the end to close the circle
        
        # Add the max scores for comparison
        max_scores_list = [max_scores[col] for col in max_score_columns]
        max_scores_list += max_scores_list[:1]  # Repeat the first score at the end to close the circle

        # Number of variables we're plotting
        num_vars = len(score_columns)

        # Compute angle each bar is centered on
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        angles += angles[:1]  # Repeat the first value to close the circle

        # Plot
        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
        
        # Draw one axe per variable + add labels
        plt.xticks(angles[:-1], columns,size=17)
        
        # Draw ylabels
        ax.set_rlabel_position(0)
        plt.yticks([], [], color="grey", size=7)
        plt.ylim(8,17)
        
        # Plot individual data
        ax.plot(angles, individual_scores, linewidth=4, linestyle='solid', label='Individual Scores')
        ax.fill(angles, individual_scores, 'b', alpha=0.1)

        # Plot average data
        ax.plot(angles, average_scores_list, linewidth=2, linestyle='dashed', label='Average Scores')

        # Plot max data
        ax.plot(angles, max_scores_list, linewidth=2, linestyle='dashed', label='Max Scores')

        # Add value labels for individual scores
        for angle, score in zip(angles[:-1], individual_scores[:-1]):
            ax.text(angle, score, str(score), color='black', size=15, horizontalalignment='center', verticalalignment='bottom')
    

        # Add a legend and a title
        plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        plt.title('Performance Radar Chart for ' + name, size=20, color='blue', y=1.1)

        plt.show()

def get_women_athelete_info(name,table=True,plot=True):
    # Select the row for the player by name
    player_row = women_individual[women_individual['name'] == name]
    print(player_row['UB_mean_score'])
    # Define the columns to keep
    columns_to_keep = [
        'all_scores_std', 'Standardized_Score_Sum', 'Total_Appearances', 
        'apparatus_participated', 'mean_score_of_all'
    ]

    player_info = player_row[columns_to_keep].T
    print(tabulate(player_info, headers='keys', tablefmt='pretty'))
    
    if table==True:
        tabular_data = {
        'Apparatus': ['Floor Exercise', 'Uneven Bar', 'Balance Beam', 'Vault'],
        'Mean Score': [player_row['FX_mean_score'].item(), player_row['UB_mean_score'].item(), 
                       player_row['BB_mean_score'].item(), player_row['VT_mean_score'].item()],
        'Standard Deviation': [player_row['FX_std'].item(), player_row['UB_std'].item(), 
                               player_row['BB_std'].item(), player_row['VT_std'].item()],
        'Appearances': [player_row['FX_count'].item(), player_row['UB_count'].item(), 
                        player_row['BB_count'].item(), player_row['VT_count'].item()],
        'Rank': [player_row['FX_rank'].item(), player_row['UB_rank'].item(), 
                 player_row['BB_rank'].item(), player_row['VT_rank'].item()]
        }
        tabular_df = pd.DataFrame(tabular_data)
        print('Statistics of', name, player_row['Country'].item())
        print(tabulate(tabular_df, headers='keys', tablefmt='pretty', showindex=False))


    if plot == True: 
        # Define the columns for mean and max scores
        score_columns = ['FX_mean_score', 'UB_mean_score', 'BB_mean_score', 'VT_mean_score']
        max_score_columns = ['FX_max_score', 'UB_max_score', 'BB_max_score', 'VT_max_score']
        columns = ['Floor Exercise', 'Uneven Bar', 'Balance Beam', 'Vault']
        
        # Extract the individual's mean scores
        individual_scores = player_row[score_columns].values.flatten().tolist()
        individual_scores += individual_scores[:1]  # Repeat the first score at the end to close the circle

        # Add the average scores for comparison
        average_scores_list = [women_average_scores[col] for col in score_columns]
        average_scores_list += average_scores_list[:1]  # Repeat the first score at the end to close the circle
        
        # Add the max scores for comparison
        max_scores_list = [women_max_scores[col] for col in max_score_columns]
        max_scores_list += max_scores_list[:1]  # Repeat the first score at the end to close the circle

        # Number of variables we're plotting
        num_vars = len(score_columns)

        # Compute angle each bar is centered on
        angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
        angles += angles[:1]  # Repeat the first value to close the circle

        # Plot
        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
        
        # Draw one axe per variable + add labels
        plt.xticks(angles[:-1], columns,size=17)
        
        # Draw ylabels
        ax.set_rlabel_position(0)
        plt.yticks([], [], color="grey", size=7)
        plt.ylim(8,17)
        
        # Plot individual data
        ax.plot(angles, individual_scores, linewidth=4, linestyle='solid', label='Individual Scores')
        ax.fill(angles, individual_scores, 'b', alpha=0.1)

        # Plot average data
        ax.plot(angles, average_scores_list, linewidth=2, linestyle='dashed', label='Average Scores')

        # Plot max data
        ax.plot(angles, max_scores_list, linewidth=2, linestyle='dashed', label='Max Scores')

        # Add value labels for individual scores
        for angle, score in zip(angles[:-1], individual_scores[:-1]):
            ax.text(angle, score, str(score), color='black', size=15, horizontalalignment='center', verticalalignment='bottom')
        

        # Add a legend and a title
        plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        plt.title('Performance Radar Chart for ' + name, size=20, color='blue', y=1.1)

        plt.show()